In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime

year_window = 15

start = f"{str(datetime.now().year-year_window)}-01-01"
end = datetime.today().strftime('%Y-%m-%d')

print(start, end)

df_market = pd.read_csv("data/Market_Stock_List.csv")

2010-01-01 2025-12-02


In [3]:
df_market.describe()

,Symbol,Company,Market,Industry,Sector,Address,Zip code,Tel.,Fax,Website
count,933,933,933,933,933,933,933,933,933,933
unique,933,933,2,9,28,920,109,888,734,908
top,ZIGA,ZIGA INNOVATION PUBLIC COMPANY LIMITED,SET,Property & Construction,-,"7-8th Floor, SCB Park Plaza 1, 18 Ratchadapise...",10120,-,-,-
freq,1,1,704,209,229,3,78,10,177,18


In [4]:
df_market.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 933 entries, 0 to 932
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Symbol    933 non-null    object
 1   Company   933 non-null    object
 2   Market    933 non-null    object
 3   Industry  933 non-null    object
 4   Sector    933 non-null    object
 5   Address   933 non-null    object
 6   Zip code  933 non-null    object
 7   Tel.      933 non-null    object
 8   Fax       933 non-null    object
 9   Website   933 non-null    object
dtypes: object(10)
memory usage: 73.0+ KB


In [5]:
df_market.nunique()

Symbol      933
Company     933
Market        2
Industry      9
Sector       28
Address     920
Zip code    109
Tel.        888
Fax         734
Website     908
dtype: int64

In [6]:
df_market["Industry"] = df_market["Industry"].replace('Industrials', 'Industrial')

In [7]:
df_ind = pd.DataFrame(df_market["Industry"].unique(), columns=["unique_values"])
df_ind.rename(columns={'unique_values': 'Industry'}, inplace=True)
df_ind['Symbol'] = None
print(df_ind.head() )

                  Industry Symbol
0               Industrial   None
1               Technology   None
2        Consumer Products   None
3  Property & Construction   None
4     Agro & Food Industry   None


In [8]:
df_all_ind = df_market.groupby("Industry")["Symbol"].apply(list).reset_index()

print(df_all_ind.head() )

                  Industry                                             Symbol
0     Agro & Food Industry  [AAI, APO, APURE, ASIAN, AU, BR, BRR, BTG, CBG...
1        Consumer Products  [88TH, AFC, AJA, APCO, AURA, BGT, BIZ, BLC, BT...
2               Financials  [ACAP, AEONTS, AF, AIRA, AMANAH, ASAP, ASK, AS...
3               Industrial  [2S, ACG, ADB, AH, AJ, AKR, ALLA, ALUCON, AMC,...
4  Property & Construction  [A, A5, AIMCG, AIMIRT, AKS, ALLY, AMATA, AMATA...


In [9]:
import yfinance as yf

for i,row in df_all_ind.iterrows():
    useable = []
    for tk in row['Symbol']:
        ticker = tk+".BK"
        data = yf.download(ticker, start=start, end=end, interval="1d", auto_adjust=True, progress=True)
        if len(data) < 2:
            continue
        data = data.dropna()
        data.index = pd.to_datetime(data.index)
        if (data.index[0].year != pd.to_datetime(start).year):
            continue
        if not data.empty:
            useable.append(tk)
    df_ind.at[i,'Symbol'] = useable
    

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [10]:
print(df_ind.head())

                  Industry                                             Symbol
0               Industrial  [APURE, ASIAN, BR, CFRESH, CHOTI, CM, CPF, CPI...
1               Technology  [AFC, BGT, BTNC, CPH, CPL, DTCI, EFORL, FANCY,...
2        Consumer Products  [ACAP, AEONTS, AF, AMANAH, ASK, ASP, AYUD, BAY...
3  Property & Construction  [2S, AH, AJ, AKR, ALUCON, AMC, BCT, BSBM, CEN,...
4     Agro & Food Industry  [A, A5, AKS, AMATA, AP, BLAND, BROCK, BSM, CCP...


In [ ]:
# df is your original DataFrame with columns "head1" and "head2" (head2 contains lists)
expanded = pd.DataFrame(df_ind['Symbol'].tolist())                 # expand lists into columns
out = pd.concat([df_ind[['Industry']].reset_index(drop=True), expanded], axis=1)
out.columns = ['Industry'] + ['Symbol'] + [''] * (expanded.shape[1] - 1)  # header: head1, head2, , , ...
out = out.fillna('')                                          # missing -> blank/None in CSV
out.to_csv('data/Industry_all.csv', index=False)

In [12]:
ls = ['^AGRO.BK', '^CONSUMP.BK', '^FINCIAL.BK', '^INDUS.BK', '^PROPCON.BK', '^RESOURC.BK', '^SERVICE.BK', '^TECH.BK']


In [13]:
all_ind = 0
no_mc = 0
no_so = 0
no_ff = 0
ls_no_mc= []
ls_no_so= []
ls_no_ff= []
for i,row in df_ind.iterrows():
    all_ind += len(row['Symbol'])
    for tk in row['Symbol']:
        try:
            now_mc = yf.Ticker(tk+'.BK').info['marketCap']
        except:
            ls_no_mc.append(tk)
            no_mc += 1

        try:
            now_so = yf.Ticker(tk+'.BK').info['sharesOutstanding']
        except:
            ls_no_so.append(tk)
            no_so += 1

        try:
            now_ff = yf.Ticker(tk+'.BK').info['floatShares']
        except:
            ls_no_ff.append(tk)
            no_ff += 1


print(all_ind)
print("NO marketCap:", no_mc, ls_no_mc)
print("NO sharesOutstanding:", no_so, ls_no_so)
print("NO free_float:", no_ff, ls_no_ff)
cant_cal = ls_no_mc + ls_no_so + ls_no_ff


434
NO marketCap: 11 ['S&J', 'WELL', 'SMT', 'SVI', 'SVOA', 'SYMC', 'SYNEX', 'TEAM', 'THCOM', 'TRUE', 'TWZ']
NO sharesOutstanding: 11 ['S&J', 'WELL', 'SMT', 'SVI', 'SVOA', 'SYMC', 'SYNEX', 'TEAM', 'THCOM', 'TRUE', 'TWZ']
NO free_float: 21 ['ACAP', 'GSTEEL', 'MIPF', 'MNIT', 'QHBREIT', 'QHOP', 'TAPAC', 'TIF1', 'TU-PF', 'SGP', 'MANRIN', 'SIS', 'SMT', 'SVI', 'SVOA', 'SYMC', 'SYNEX', 'TEAM', 'THCOM', 'TRUE', 'TWZ']


In [14]:
print(cant_cal)

['S&J', 'WELL', 'SMT', 'SVI', 'SVOA', 'SYMC', 'SYNEX', 'TEAM', 'THCOM', 'TRUE', 'TWZ', 'S&J', 'WELL', 'SMT', 'SVI', 'SVOA', 'SYMC', 'SYNEX', 'TEAM', 'THCOM', 'TRUE', 'TWZ', 'ACAP', 'GSTEEL', 'MIPF', 'MNIT', 'QHBREIT', 'QHOP', 'TAPAC', 'TIF1', 'TU-PF', 'SGP', 'MANRIN', 'SIS', 'SMT', 'SVI', 'SVOA', 'SYMC', 'SYNEX', 'TEAM', 'THCOM', 'TRUE', 'TWZ']


In [15]:
df_cut_ind = pd.DataFrame(df_market["Industry"].unique(), columns=["unique_values"])
df_cut_ind.rename(columns={'unique_values': 'Industry'}, inplace=True)
df_cut_ind['Symbol'] = None
print(df_cut_ind.head())

                  Industry Symbol
0               Industrial   None
1               Technology   None
2        Consumer Products   None
3  Property & Construction   None
4     Agro & Food Industry   None


In [30]:
# inputs:
# prices: DataFrame indexed by date (monthly), columns = tickers (close price)
# shares: Series indexed by tickers (sharesOutstanding)
# free_float: Series indexed by tickers (free-float factor, decimal)
# M = lookback months, coverage_target = 0.95

c_v = 0.90
K = 10

def compute_smoothed_ffmc(prices, shares, free_float, M=3):
    # prices: DataFrame monthly closes
    const = shares * free_float             # Series length N
    # print(const.head())
    ffmc = prices.mul(const, axis=1)        # DataFrame (dates x tickers)
    # rolling mean over last M rows (align to right)
    ffmc_smoothed = ffmc.rolling(window=M, min_periods=1).mean()
    return ffmc_smoothed

def select_top_k_by_coverage(ffmc_smoothed, date, coverage_target=c_v):
    ffmc_today = ffmc_smoothed.loc[date].dropna()
    total = ffmc_today.sum()
    weights = ffmc_today / total
    weights_sorted = weights.sort_values(ascending=False)
    cumsum = weights_sorted.cumsum()
    # smallest K achieving coverage_target
    topk = weights_sorted[cumsum <= coverage_target].index.tolist()
    # if none selected (coverage_target too high), take first K default:
    if len(topk) == 0:
        topk = weights_sorted.head(K).index.tolist()
    # ensure coverage >= target (add next until >=)
    while weights_sorted[topk].sum() < coverage_target and len(topk) < len(weights_sorted):
        next_idx = weights_sorted.index[len(topk)]
        topk.append(next_idx)
    if (len(topk) > K):
        topk = topk[:K]
    return topk, weights_sorted

In [31]:
cov_col = {}
for i,row in df_ind.iterrows():
    now_ind = df_ind.iloc[i,0]
    col = []
    prices = pd.DataFrame()
    shares = pd.Series(dtype=float)
    free_float = pd.Series(dtype=float)
    for tk in row['Symbol']:
        if tk in cant_cal:
            continue
        t = tk+'.BK'
        df_daily = yf.download(t, start=start, end=end, interval="1d", auto_adjust=True, progress=True)
        if len(df_daily) < 2:
            continue
        if "Adj Close" in df_daily.columns:
            price_daily = df_daily["Adj Close"].copy()
        elif "Close" in df_daily.columns:
            price_daily = df_daily["Close"].copy()
        else:
            price_daily = df_daily.iloc[:, 3].copy()

        price_daily = price_daily.dropna()
        price_daily.index = pd.to_datetime(price_daily.index)

        if (price_daily.index[0].year != pd.to_datetime(start).year):
            continue

        price_monthly = price_daily.resample('ME').last().dropna()

        now_so = yf.Ticker(t).info['sharesOutstanding']
        now_fs = yf.Ticker(t).info['floatShares']
        now_ff = now_fs/now_so

        if (prices.empty):
            prices = price_monthly.copy()
        else:
            prices = pd.concat([prices, price_monthly], axis=1)
        shares[t] = float(now_so)
        free_float[t] = float(now_ff)


    # Example usage
    ffmc_smoothed = compute_smoothed_ffmc(prices, shares, free_float, M=3)
    date = ffmc_smoothed.index[-1]   # latest month
    topk, weights_sorted = select_top_k_by_coverage(ffmc_smoothed, date)
    coverage = weights_sorted[topk].sum()
    # print(shares.head())
    # print(f"Top-K tickers for {now_ind}: {topk}")
    # print("Coverage:", coverage)
    df_cut_ind.at[i,'Symbol'] = topk
    cov_col[now_ind] = coverage
    
    
        

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [32]:
from pprint import pprint
pprint(cov_col)
print(df_cut_ind.head())
print(df_cut_ind.tail())

{'Agro & Food Industry': np.float64(0.8360518122727917),
 'Consumer Products': np.float64(0.9217599652983842),
 'Financials': np.float64(0.9429481221882371),
 'Industrial': np.float64(0.9000206863683722),
 'Property & Construction': np.float64(0.6504641597411253),
 'Resources': np.float64(0.877355847432804),
 'Services': np.float64(0.9074653119414025),
 'Technology': np.float64(0.8588338566949874)}
                  Industry                                             Symbol
0               Industrial  [CPF.BK, TU.BK, TFMAMA.BK, TVO.BK, STA.BK, GFP...
1               Technology  [SUC.BK, SABINA.BK, TOG.BK, KYE.BK, MODERN.BK,...
2        Consumer Products  [KBANK.BK, BBL.BK, KTB.BK, TISCO.BK, TTB.BK, K...
3  Property & Construction  [IVL.BK, NEX.BK, MCS.BK, BCT.BK, SAT.BK, STANL...
4     Agro & Food Industry  [CPN.BK, SCC.BK, LH.BK, CPNREIT.BK, SIRI.BK, A...
                  Industry                                             Symbol
3  Property & Construction  [IVL.BK, NEX.BK, MCS.BK,

In [36]:
cut = []
for i in cov_col:
    if cov_col[i] < c_v:
        cut.append(i)
print(cut)

['Technology', 'Property & Construction', 'Agro & Food Industry', 'Resources']


In [33]:
# df is your original DataFrame with columns "head1" and "head2" (head2 contains lists)
expanded = pd.DataFrame(df_cut_ind['Symbol'].tolist())                 # expand lists into columns
out = pd.concat([df_cut_ind[['Industry']].reset_index(drop=True), expanded], axis=1)
out.columns = ['Industry'] + ['Symbol'] + [''] * (expanded.shape[1] - 1)  # header: head1, head2, , , ...
out = out.fillna('')                                          # missing -> blank/None in CSV
out.to_csv('data/Industry_sorted.csv', index=False)